<a href="https://colab.research.google.com/github/DJLee68/2020_cau_oss_hackathon/blob/master/hackathon_team2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 5시 (단, 발표자료는 11시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2020_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 테스트 데이터 셋 분류 정확도
 *  model.evaluate(x_test, y_test)
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드 or 알고리즘이 적발될 경우
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

import numpy as np

# 데이터셋 다운로드
check = !if [ -d 'dataset/' ]; then echo "1" ; else echo "0"; fi
if (check[0] is '0' ):
  !mkdir dataset
  !wget 'https://www.itl.nist.gov/iaui/vip/cs_links/EMNIST/matlab.zip'
  !unzip matlab.zip -d /content/dataset

# 데이터셋 로드
from scipy import io as spio
emnist = spio.loadmat("/content/dataset/matlab/emnist-balanced.mat")

x_train = emnist["dataset"][0][0][0][0][0][0]
y_train = emnist["dataset"][0][0][0][0][0][1]

x_test = emnist["dataset"][0][0][1][0][0][0]
y_test = emnist["dataset"][0][0][1][0][0][1]

# # 분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# 데이터 28x28 이미지화
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32')

# 총 클래스 개수
num_classes = y_test.shape[1]
input_shape = x_test.shape[1:]

# **2. 데이터 전처리**



In [13]:
# 데이터 전처리 (예: normalization)
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0

In [14]:
class_num = y_train.shape[1]

# **3. 모델 생성**

In [15]:
from keras.models import load_model, Model, Input, Sequential
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPool2D, ReLU, Dropout, Flatten, GlobalAveragePooling2D
from keras.backend import sigmoid
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.efficientnet import EfficientNetB1

In [ ]:
# 아무래도 Imagenet data로 high accuracy가 입증이 된, VGG16, EfficientNet 등의 pretrained model을 가져와 학습하면 좋은 성능으로 이어지리라 생각을 했습니다.

# 이번 해커톤의 데이터는 Imagenet data가 아니라, EMNIST 데이터이기에 모델 형식만 가져오고 scratch 부터 학습을 시켰습니다. 
# Scratch부터 훈련시킨 이유는 Imagenet data는 우선 크기도 훨씬 크고, RGB(3 channel)이고 image안에 여러 object가 있을 수 있는 반면,
# Gray-scale(1 channel)이고 크기고 작고 object가 하나 밖에없는 EMNIST와는 달라서 다시 학습이 필요하다고 생각했기 때문입니다.
# 그럼에도 불구하고 model의 형태 자체가 워낙 발전되고 입증된 모델이어서 좋은 성능을 기대헀는데, 제 기대와는 달리 accuracy가 높지 않았습니다.

# Keras의 Imagedatagenerator 클래스를 통해 augmentation도 해봤는데(EMNIST 데이터를 직접 확인하니까, 약간 기울어져있는 것도 있었고 flip된 이미지도 보였습니다) 별 도움이 안되었고,
# Scaling된 데이터들도 보이길래 FFT(Scale invariance를 가진)를 적용시켜서 훈련시켜볼 까 생각도 했지만, 득보단 실이 클것 같아서 하지 않았습니다.

# 결국 그냥 단순하게 vgg 형식의 conv block(conv-batchnorm-activation)을 쌓는 것으로 방향을 바꿨는데 오히려 이게 더 성능이 좋아서, 이것으로 제출합니다.


In [16]:
model =  keras.Sequential([
    Conv2D(64, kernel_size = 3, strides = 1, padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    ReLU(),
    Conv2D(64, kernel_size = 3, strides=1, padding='same'),
    BatchNormalization(),
    ReLU(),
    MaxPool2D(pool_size=(2,2), strides=2),

    Conv2D(128, kernel_size = 3, strides = 1, padding='same'),
    BatchNormalization(),
    ReLU(),
    Conv2D(128, kernel_size = 3, strides=1, padding='same'),
    BatchNormalization(),
    ReLU(),
    Conv2D(128, kernel_size = 3, strides=1, padding='same'),
    BatchNormalization(),
    ReLU(),
    MaxPool2D(pool_size=(2,2), strides=2),

    Conv2D(256, kernel_size = 3, strides = 1, padding='same'),
    BatchNormalization(),
    ReLU(),
    Conv2D(256, kernel_size = 3, strides=1, padding='same'),
    BatchNormalization(),
    ReLU(),
    Conv2D(256, kernel_size = 3, strides=1, padding='same'),
    BatchNormalization(),
    ReLU(),
    MaxPool2D(pool_size=(3,3), strides=2),
    Flatten(),
    Dense(1024, activation = 'relu'),
    Dropout(0.2),
    Dense(512, activation = 'relu'),
    Dense(47, activation = 'softmax')
])

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_36 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
re_lu_36 (ReLU)              (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_37 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
re_lu_37 (ReLU)              (None, 28, 28, 64)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 64)       

In [17]:
# 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set
model.fit(x_train_after, y_train, batch_size = 128, epochs = 50, shuffle=True, callbacks=[cp_callback], validation_data=(x_test_after, y_test))

Epoch 1/50
882/882 [==============================] - ETA: 0s - loss: 0.9657 - accuracy: 0.7136
Epoch 00001: val_accuracy improved from -inf to 0.83904, saving model to /content/checkpoint_entire_best.h5
882/882 [==============================] - 34s 39ms/step - loss: 0.9657 - accuracy: 0.7136 - val_loss: 0.4677 - val_accuracy: 0.8390
Epoch 2/50
881/882 [============================>.] - ETA: 0s - loss: 0.3844 - accuracy: 0.8622
Epoch 00002: val_accuracy improved from 0.83904 to 0.84840, saving model to /content/checkpoint_entire_best.h5
882/882 [==============================] - 33s 38ms/step - loss: 0.3845 - accuracy: 0.8622 - val_loss: 0.4231 - val_accuracy: 0.8484
Epoch 3/50
881/882 [============================>.] - ETA: 0s - loss: 0.3421 - accuracy: 0.8756
Epoch 00003: val_accuracy improved from 0.84840 to 0.86245, saving model to /content/checkpoint_entire_best.h5
882/882 [==============================] - 33s 37ms/step - loss: 0.3422 - accuracy: 0.8756 - val_loss: 0.3947 - val_

# **4. 모델 저장**

In [18]:
save_path = '/content/'
team_name = 'team2'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [19]:
save_path = '/content/'
team_name = 'team2'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test_after, y_test)

588/588 [==============================] - 3s 5ms/step - loss: 0.7968 - accuracy: 0.8866


[0.7967966198921204, 0.8865957260131836]

In [20]:
save_path = '/content/'
team_name = 'team2'

model = keras.models.load_model(save_path + 'checkpoint_entire_best' + '.h5')

model.evaluate(x_test_after, y_test)

588/588 [==============================] - 3s 5ms/step - loss: 0.2989 - accuracy: 0.9005


[0.2988581657409668, 0.9005318880081177]